# SDS to website
Purpose: By processing a list of events from an Excel spreadsheet, look for existing seismic & infrasound data in an SDS archive, instrument correct those data, and make a webpage for each event 

Steps:
1. Create an event catalog with instrument-corrected data

TO DO: 
* add support for AM network, and any data from networks run by other operators and available on IRIS
* add well data from YYYYMMDD_final.pkl files

Do we want to turn pkl files into SDS?

SCAFFOLD:
* add a detector for sonic booms after rocket launch for landers
* add any nearby IRIS seismic and infrasound data

# 1. Do full workflow for all events
* Load and correct seismic and infrasound data from FL network and XA and whatever the network was for PASSCAL equipment

In [1]:
import os
import glob
import sys
import header
paths = header.setup_environment()
paths['SDS_TOP'] = '/data/SDS'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import obspy
sys.path.append(os.path.join(paths['Developer'], 'SoufriereHillsVolcano', 'lib'))
import SDS
#import Spectrograms
import USF_instrument_responses
from obspy.clients.filesystem.sds import Client
#f#rom obspy.signal.cross_correlation import correlate, xcorr_max
#from obspy.signal.trigger import 
from pprint import pprint
import matplotlib.dates as dates
import libseisGT
import libWellData as LLE
import librockets as LR

sdsclient = Client(paths['SDS_TOP'])
thisSDSobj = SDS.SDSobj(paths['SDS_TOP'])

paths['EVENTS'] = os.path.join(paths['DROPBOX_DATA_TOP'], 'EVENTS')

paths['WWW_TOP'] = '/var/www/html/usfseismiclab.org/html/rocketcat'
paths['WWW_EVENTS']=os.path.join(paths['WWW_TOP'], 'EVENTS')
#paths['WWW_CONTINUOUS'] = os.path.join(paths['WWW_TOP'], 'CONTINUOUS')



Linux
Linux


### Setup for 2022 EROSION PROJECT

In [2]:
if False:
    paths['csv_launches'] = os.path.join(paths['WWW_TOP'], 'launches.csv')

    launchesDF = LLE.removed_unnamed_columns(pd.read_csv(paths['csv_launches'], index_col=None, parse_dates=['datetime']))
    #launchesDF['datetime']= pd.to_datetime(launchesDF['Date'] + ' ' +  launchesDF['Time'])
    preseconds=120
    eventseconds=120
    postseconds=120
    taperseconds=600

    # need to add a detector for sonic booms too!
    # 2022 responses
    ondate = obspy.UTCDateTime(2022,3,17)
    offdate = obspy.UTCDateTime(2022,12,5)

    # seismic channels
    invs1 = USF_instrument_responses.NRL2inventory('FL', 'S39A1', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs2 = USF_instrument_responses.NRL2inventory('FL', 'S39A2', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs3 = USF_instrument_responses.NRL2inventory('FL', 'S39A3', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs4 = USF_instrument_responses.NRL2inventory('FL', 'BCHH3', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs5 = USF_instrument_responses.NRL2inventory('FL', 'BCHH4', '10', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    seismicinv = invs1 + invs2 + invs3 + invs4 + invs5

    # infrasound
    date_BCHH3_change = obspy.UTCDateTime(2022,5,26) # looking at files, it seems we actually have no data for BCHH3 after this - we just called it BCHH4
    invi1 = USF_instrument_responses.NRL2inventory('FL', 'S39A1', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi2 = USF_instrument_responses.NRL2inventory('FL', 'S39A2', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi3 = USF_instrument_responses.NRL2inventory('FL', 'S39A3', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi4 = USF_instrument_responses.NRL2inventory('FL', 'BCHH2', '10', ['HD4'], datalogger='Centaur', sensor='Chap', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi5 = USF_instrument_responses.NRL2inventory('FL', 'BCHH2', '10', ['HD5', 'HD6'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi6 = USF_instrument_responses.NRL2inventory('FL', 'BCHH2', '10', ['HD7', 'HD8', 'HD9'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi7 = USF_instrument_responses.NRL2inventory('FL', 'BCHH3', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=date_BCHH3_change)
    invi8 = USF_instrument_responses.NRL2inventory('FL', 'BCHH3', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=100.0, ondate=date_BCHH3_change, offdate=offdate)
    invi9 = USF_instrument_responses.NRL2inventory('FL', 'BCHH4', '00', ['HDF', 'HD2', 'HD3'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)

    infrainv = invi1 + invi2 + invi3 + invi4 + invi5 + invi6 + invi7 + invi8 + invi9

    inv = seismicinv + infrainv


### Setup for whole SDS archive of rocket launches

In [3]:
import datetime
# 1. Load and modify the all launches Excel file
paths['xls_launches'] = os.path.join('/data', 'KSC','All_KSC_Rocket_Launches.xlsx')

launchesDF = LLE.removed_unnamed_columns(pd.read_excel(paths['xls_launches'], index_col=None, parse_dates=['Date']))
launchesDF = launchesDF[['Date', 'Time', 'SLC', 'Rocket_Payload', 'Event type', 'Returned to KSC']]
#launchesDF['datetime']= pd.to_datetime(launchesDF['Date'] + ' ' +  launchesDF['Time'])

try:
    time_as_timedelta = pd.to_timedelta(launchesDF['Time'])
except:
    time_series = launchesDF['Time']
    time_as_timedelta = pd.to_timedelta([datetime.timedelta(hours=t.hour, minutes=t.minute, seconds=t.second) for t in time_series])


# Combine the date and time series into a single datetime column
datetime_series = launchesDF['Date'] + time_as_timedelta

# Convert to obspy UTCDateTime format
launchesDF['datetime'] = datetime_series.apply(lambda x: obspy.UTCDateTime(x))
del datetime_series

print(launchesDF)


          Date      Time  SLC                    Rocket_Payload Event type  \
0   2016-02-05  13:38:00   41       Atlas V 401|AV057 GPS 2F-12     launch   
1   2016-03-04  23:35:00   40          Falcon 9 1.2|F9-22 SES-9     launch   
2   2016-03-23  03:05:52   41     Atlas V 401|AV064 Cygnus OA-6     launch   
3   2016-04-08  20:43:31   40  Falcon 9 1.2|F9-023 Dragon/CRS 8     launch   
4   2016-05-06  05:21:00   40       Falcon 9 1.2|F9-24 JCSAT 14     launch   
..         ...       ...  ...                               ...        ...   
180 2022-11-03  05:22:00   40         Falcon 9 Block 5 | F9-184        NaN   
181 2022-11-12  16:06:00   40         Falcon 9 Block 5 | F9-185        NaN   
182 2022-11-16  06:47:44  39B          SLS Block 1 | Artemis 1         NaN   
183 2022-11-23  02:57:00   40         Falcon 9 Block 5 | F9-186        NaN   
184 2022-11-26  19:20:43  39A         Falcon 9 Block 5 | F9-187        NaN   

    Returned to KSC                     datetime  
0           

In [4]:
preseconds=120
eventseconds=120
postseconds=120
taperseconds=600

# need to add a detector for sonic booms too!
# 2022 responses
ondate = obspy.UTCDateTime(2016, 2, 24)
offdate = obspy.UTCDateTime(2022,12,5)

xmlfile = 'KSC.xml'
if os.path.isfile(xmlfile):
    inv = obspy.core.inventory.read_inventory(xmlfile)
else:

    # seismic channels
    invs1 = USF_instrument_responses.NRL2inventory('FL', 'S39A1', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs2 = USF_instrument_responses.NRL2inventory('FL', 'S39A2', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs3 = USF_instrument_responses.NRL2inventory('FL', 'S39A3', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs4 = USF_instrument_responses.NRL2inventory('FL', 'BCHH3', '00', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invs5 = USF_instrument_responses.NRL2inventory('FL', 'BCHH4', '10', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    seismicinv = invs1 + invs2 + invs3 + invs4 + invs5

    # infrasound
    date_BCHH3_change = obspy.UTCDateTime(2022,5,26) # looking at files, it seems we actually have no data for BCHH3 after this - we just called it BCHH4
    invi1 = USF_instrument_responses.NRL2inventory('FL', 'S39A1', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi2 = USF_instrument_responses.NRL2inventory('FL', 'S39A2', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi3 = USF_instrument_responses.NRL2inventory('FL', 'S39A3', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi4 = USF_instrument_responses.NRL2inventory('FL', 'BCHH2', '10', ['HD4'], datalogger='Centaur', sensor='Chap', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi5 = USF_instrument_responses.NRL2inventory('FL', 'BCHH2', '10', ['HD5', 'HD6'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi6 = USF_instrument_responses.NRL2inventory('FL', 'BCHH2', '10', ['HD7', 'HD8', 'HD9'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)
    invi7 = USF_instrument_responses.NRL2inventory('FL', 'BCHH3', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=date_BCHH3_change)
    invi8 = USF_instrument_responses.NRL2inventory('FL', 'BCHH3', '10', ['HDF'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=100.0, ondate=date_BCHH3_change, offdate=offdate)
    invi9 = USF_instrument_responses.NRL2inventory('FL', 'BCHH4', '00', ['HDF', 'HD2', 'HD3'], datalogger='Centaur', sensor='infraBSU', Vpp=1, fsamp=100.0, ondate=ondate, offdate=offdate)

    infrainv = invi1 + invi2 + invi3 + invi4 + invi5 + invi6 + invi7 + invi8 + invi9


    for station_num in range(1,9,1):
        # 2018-2021
        station = f'BHP{station_num}'
        invpasscal = USF_instrument_responses.NRL2inventory('1R', station, '', ['EH1', 'EH2', 'EHZ'], datalogger='RT130', sensor='L-22', fsamp=100.0, ondate=ondate, offdate=offdate)
        seismicinv = seismicinv + invpasscal
    for station in ['FIREP', 'TANKP']:
        invpasscal = USF_instrument_responses.NRL2inventory('1R', station, '', ['EH1', 'EH2', 'EHZ'], datalogger='RT130', sensor='L-22', fsamp=100.0, ondate=ondate, offdate=offdate)
        seismicinv = seismicinv + invpasscal

    inv0 = USF_instrument_responses.NRL2inventory('FL', 'BCHH1', '0', ['HHZ', 'HHN', 'HHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    seismicinv = seismicinv + inv0
    inv1 = USF_instrument_responses.NRL2inventory('FL', 'BCHH1', '0', ['HD1', 'HD2', 'HD3'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=100.0, ondate=ondate, offdate=offdate)
    infrainv = infrainv + inv1

    inv2 = USF_instrument_responses.NRL2inventory('FL', 'BCHH', '00', ['GHZ', 'GHN', 'GHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=2000.0, ondate=ondate, offdate=offdate)
    seismicinv = seismicinv + inv2
    inv3 = USF_instrument_responses.NRL2inventory('FL', 'BCHH', '10', ['GD1', 'GD2', 'GD3'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=2000.0, ondate=ondate, offdate=offdate)
    infrainv = infrainv + inv3

    for station in ['BCHH', 'FIRE', 'TANK']:
        inv4 = USF_instrument_responses.NRL2inventory('FL', station, '00', ['DHZ', 'DHN', 'DHE'], datalogger='Centaur', sensor='TCP', Vpp=40, fsamp=250.0, ondate=ondate, offdate=offdate)
        seismicinv = seismicinv + inv4
        inv5 = USF_instrument_responses.NRL2inventory('FL', station, '10', ['DD1', 'DD2', 'DD3'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=250.0, ondate=ondate, offdate=offdate)
        infrainv = infrainv + inv5

    inv6 = USF_instrument_responses.NRL2inventory('FL', 'BCHH2', '10', ['HD4', 'HD5', 'HD6', 'HD7', 'HD8', 'HD9'], datalogger='Centaur', sensor='infraBSU', Vpp=40, fsamp=250.0, ondate=ondate, offdate=offdate)
    infrainv = infrainv + inv6


    inv = seismicinv + infrainv

    inv.write(xmlfile, format='STATIONXML')

In [6]:
df = launchesDF
start_date = obspy.UTCDateTime(2016, 2, 20, 0, 0, 0)
end_date = obspy.UTCDateTime(2016, 9, 10, 23, 59, 59)
subset_df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]


for idx, eventrow in subset_df.iterrows():
    # get start and end time. remember that spreadsheet is in UTC. well data are in local time but summer time is assumed throughout (UTC-4)
    launchtimeUTC = eventrow['datetime']

    # create event directory
    EVENTDIR = os.path.join(paths['WWW_EVENTS'], launchtimeUTC.isoformat())
    if not os.path.isdir(EVENTDIR):
        os.makedirs(EVENTDIR)  

    print('Processing launch at %s' % launchtimeUTC.strftime('%Y-%m-%d %H:%M:%S')) 
    st = LR.sds2eventStream(launchtimeUTC, sdsclient, thisSDSobj, \
                            pretrig=preseconds+taperseconds, \
                                posttrig=eventseconds+postseconds+taperseconds,
                                networks = ['FL', '1R'])

    print(st)

    continue
    # Define the station you want to remove (e.g., 'STATION_NAME')
    station_to_remove = "DWPF"
    #st = st.select(lambda tr: tr.stats.station != station_to_remove)
    for tr in st:
        if tr.stats.station == station_to_remove:
            st.remove(tr)
        elif tr.data.size == 0 or all(tr.data == 0):
            print(f'- removing {tr.id}')
            st.remove(tr)
    if len(st)==0:
        print('- no Trace objects')
        continue
    

    #st = st.select(network='FL') # while AM, XA, and 1R are also valid, we need to get instrument response data for them

    #st = st.filter(lambda trace: trace.data.size > 0 and not all(trace.data == 0))

    # all these functions safe for well traces too - if they were in the Stream objects as they were when we converted well data to SDS
    
    st.plot(equal_scale=False, outfile=os.path.join(EVENTDIR, 'raw.png'));
    #LR.clean(st, taperseconds) # despike, detrend, taper, high pass filter, trim
    for tr in st:
        trace_seconds = tr.stats.delta * tr.stats.npts
        libseisGT.clean_trace(tr, taperFraction=taperseconds/trace_seconds, filterType="highpass", freq=0.01, corners=2, zerophase=True, inv=None)
    
    if len(st)==0:
        print('- no Trace objects')
        continue
    infrasound_st = st.select(channel='[ESBHDG]D?')
    #apply_calibration_correction(infrasound_st) 
    USF_instrument_responses.correctUSFstations(infrasound_st, apply_calib=True, attach=False, return_inventories=False)
    #libseisGT.removeInstrumentResponse(infrasound_st, preFilter = None, outputType = "DISP", inventory = infrainv)    
    LR.detect_incorrect_p2p_voltage_setting(infrasound_st)

    seismic_st = st.select(channel='[ESBHDG]H?')
    velocity_st = seismic_st.copy()
    #velocity_st = remove_response(seismic_st, output='VEL')
    libseisGT.removeInstrumentResponse(velocity_st, preFilter = None, outputType = "VEL", inventory = inv)
    LR.detect_incorrect_p2p_voltage_setting(velocity_st) 

    # write corrected event out
    #correctedfile =  os.path.join(EVENTDIR, '%s.pkl' % launchtimeUTC.strftime('%Y%m%dT%H%M%S'))
    #st.write(correctedfile, format='PICKLE') 

    best_trig = LR.detectEvent(infrasound_st + velocity_st) # + displacement_st)
    pad_secs = 10
    trigtime = launchtimeUTC
    duration = eventseconds
    if len(best_trig)>0:    
        trigtime = best_trig['time']
        duration = best_trig['duration']
    trimtime0 = LR.floor_minute(trigtime-pad_secs)
    trimtime1 = LR.ceil_minute(trigtime+duration+pad_secs)
    infrasound_st.trim(starttime=trimtime0, endtime=trimtime1)
    velocity_st.trim(starttime=trimtime0, endtime=trimtime1)
    displacement_st = seismic_st.copy()
    #displacement_st = remove_response(seismic_st, output='DISP')
    libseisGT.removeInstrumentResponse(displacement_st, preFilter = None, outputType = "DISP", inventory = inv)
    LR.detect_incorrect_p2p_voltage_setting(displacement_st) 
    displacement_st.trim(starttime=trimtime0, endtime=trimtime1)

    # SCAFFOLD
    # generate seismic and infrasound figures - but change to add RSAM pandas plots, spectrograms and spectra, etc.
    '''
    fig, ax = plt.subplots(figsize=(10, 6))  # Set a custom figure size (width, height)
    infrasound_st.plot(fig=fig, equal_scale=False, outfile=os.path.join(EVENTDIR, 'infrasound.png'));
    fh,axh=libseisGT.mulplt(infrasound_st, bottomlabel='', ylabels=[], MAXPANELS=6)
    fh.savefig(fname=os.path.join(EVENTDIR, 'infrasound2.png'))
    '''
    #super_stream_plot(velocity_st, dpi=100, Nmax=6, rank='ZFODNE123456789', equal_scale=False, outfile=os.path.join(EVENTDIR, 'velocity.png'))
    #super_stream_plot(displacement_st, dpi=100, Nmax=6, rank='ZFODNE123456789', equal_scale=False, outfile=os.path.join(EVENTDIR, 'displacement.png'))
    if len(infrasound_st)>0:
        print('- plotting infrasound')
        LR.plot_seismograms(infrasound_st, outfile=os.path.join(EVENTDIR, 'infrasound.png'), bottomlabel=None, ylabels=None, units='Pa', channels='F123456789')
    if len(velocity_st)>0:
        print('- plotting seismic')
        LR.plot_seismograms(velocity_st, outfile=os.path.join(EVENTDIR, 'velocity.png'), bottomlabel=None, ylabels=None, units='m/s')
        LR.plot_seismograms(displacement_st, outfile=os.path.join(EVENTDIR, 'displacement.png'), bottomlabel=None, ylabels=None, units='m')
    #plot_envelope2(infrasound_st, outfile=os.path.join(EVENTDIR, 'infrasound5.png'), units='Pa')
    #plot_envelope2(velocity_st.select(channel='HHZ'), outfile=os.path.join(EVENTDIR, 'velocity2.png'), units='m/s')
    #plot_envelope2(displacement_st.select(channel='HHZ'), outfile=os.path.join(EVENTDIR, 'displacement2.png'), units='m')
    '''
    fig, ax = plt.subplots(figsize=(10, 6))  # Set a custom figure size (width, height)
    velocity_st.plot(fig=fig, equal_scale=False, outfile=os.path.join(EVENTDIR, 'velocity.png'));
    fig, ax = plt.subplots(figsize=(10, 6))  # Set a custom figure size (width, height)
    displacement_st.plot(fig=fig, equal_scale=False, outfile=os.path.join(EVENTDIR, 'displacement.png'));    
    # Optionally, set the aspect ratio (e.g., 2:1 aspect ratio)
    #ax.set_aspect(2.0)
    '''

Processing launch at 2016-03-04 23:35:00
FL/
    BCHH/
        00/GHE
        00/GHN
        00/GHZ
        10/GD1
        10/GD2
        10/GD3
    FIRE/
        00/DHE
        00/DHN
        00/DHZ
        10/DD1
        10/DD2
        10/DD3
NZ/
    WIZ/
        10/HHE
        10/HHN
        10/HHZ
        31/HDF
        32/HDF
    WSRZ/
        10/HHE
        10/HHN
        10/HHZ
        31/HDF
        32/HDF
12 Trace(s) in Stream:
FL.BCHH.00.GHE | 2016-03-04T23:23:00.000000Z - 2016-03-04T23:49:00.000000Z | 2000.0 Hz, 3120001 samples
FL.BCHH.00.GHN | 2016-03-04T23:23:00.000000Z - 2016-03-04T23:49:00.000000Z | 2000.0 Hz, 3120001 samples
FL.BCHH.00.GHZ | 2016-03-04T23:23:00.000000Z - 2016-03-04T23:49:00.000000Z | 2000.0 Hz, 3120001 samples
FL.BCHH.10.GD1 | 2016-03-04T23:23:00.000000Z - 2016-03-04T23:49:00.000000Z | 2000.0 Hz, 3120001 samples
FL.BCHH.10.GD2 | 2016-03-04T23:23:00.000000Z - 2016-03-04T23:49:00.000000Z | 2000.0 Hz, 3120001 samples
FL.BCHH.10.GD3 | 2016-03-04T23:23:00.00